# Prepare Sankey Data

In [1]:
from tqdm.notebook import tqdm
from concurrent import futures

import itertools
import pandas as pd
import numpy as np

In [2]:
subset = pd.read_csv('alum-edu-hist.csv')
subset

/home/agupta/Documents/education/wpi/senior/iqp/career-trajectory-mapping/venv-iqp/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Contact: Case Safe ID,Contact: Full Name,Contact: Constituent Category (Primary),Contact: Preferred Class Year,Education: Education History,Degree/Certificate,Degree Year,End Year,Graduation Year,Institution Name,...,Minor #2,Minor #2 Department,Other Information Related to Degree,Start Year,Student Status,Education: Record Type,Education: Created By,Education: Created Date,Education: Last Modified By,Education: Last Modified Date
0,003G0000024bMwNIAU,Mark Korir,Graduate Alumnus,2017.0,120338,Master of Business Admin.,2014.0,NaN,2014.0,Hodges University,...,NaN,NaN,NaN,NaN,NaN,Non-WPI Degree,Anthony McMahon,8/23/2018,Anthony McMahon,8/23/2018
1,003G000001oerenIAA,Rebecca Ide,Alumna/us,2012.0,185152,Non-Degree,NaN,NaN,NaN,Becker College Worcester,...,NaN,NaN,NaN,NaN,NaN,WPI Degree,API-Only,8/27/2018,API-Only,8/27/2018
2,003G000001of21qIAA,Kevin Lynch,Alumna/us,2018.0,174184,Bachelor of Science,2017.0,NaN,2017.0,Worcester Polytechnic Institute,...,NaN,NaN,NaN,NaN,Inactive,WPI Degree,Anthony McMahon,8/23/2018,Jeffrey Caglarcan,8/18/2019
3,003G000001oeYxWIAU,Zareh Martin,Alumna/us,1940.0,155692,Bachelor of Science,1940.0,NaN,1940.0,Worcester Polytechnic Institute,...,NaN,NaN,NaN,NaN,Inactive,WPI Degree,Anthony McMahon,8/23/2018,Jeffrey Caglarcan,8/18/2019
4,003G000001oeaVGIAY,Gerald Orre,Alumna/us,1971.0,158957,Bachelor of Science,1971.0,NaN,1971.0,Worcester Polytechnic Institute,...,NaN,NaN,NaN,NaN,Inactive,WPI Degree,Anthony McMahon,8/23/2018,Jeffrey Caglarcan,8/18/2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80709,003G000002fezPIIAY,Shikhar Gupta,Graduate Alumnus,2018.0,141101,Other Bachelors Degree,NaN,2012.0,NaN,SIES College of Mgt Studies,...,NaN,NaN,NaN,2009.0,NaN,Non-WPI Degree,Anthony McMahon,8/23/2018,Anthony McMahon,8/23/2018
80710,003G000002fezPIIAY,Shikhar Gupta,Graduate Alumnus,2018.0,181530,Other Bachelors Degree,2012.0,NaN,2012.0,SIES College of Mgt Studies,...,NaN,NaN,NaN,2017.0,Inactive due to graduation,Non-WPI Degree,API-Only,8/24/2018,API-Only,3/25/2019
80711,003G000002Fh4xzIAB,Voon Li Koh,Graduate Alumnus,2020.0,142125,Bachelor of Engineering,NaN,2009.0,NaN,Universiti Tunku Abdul Rahman,...,NaN,NaN,NaN,2006.0,NaN,Non-WPI Degree,Anthony McMahon,8/23/2018,Anthony McMahon,8/23/2018
80712,003G000001of33gIAA,Zhaolong Zhang,Graduate Alumnus,2016.0,109269,Other Graduate Degree,2012.0,NaN,2012.0,Pittsburg State University,...,NaN,NaN,NaN,NaN,NaN,Non-WPI Degree,Anthony McMahon,8/23/2018,Anthony McMahon,8/23/2018


In [3]:
subset.columns

Index(['Contact: Case Safe ID', 'Contact: Full Name',
       'Contact: Constituent Category (Primary)',
       'Contact: Preferred Class Year', 'Education: Education History',
       'Degree/Certificate', 'Degree Year', 'End Year', 'Graduation Year',
       'Institution Name', 'Major #1', 'Major #1 Department', 'Major #2',
       'Major #2 Department', 'Minor #1', 'Minor #1 Department', 'Minor #2',
       'Minor #2 Department', 'Other Information Related to Degree',
       'Start Year', 'Student Status', 'Education: Record Type',
       'Education: Created By', 'Education: Created Date',
       'Education: Last Modified By', 'Education: Last Modified Date'],
      dtype='object')

In [4]:
all_majors = pd.concat((subset['Major #1'], subset['Major #2'])).unique()
all_minors = pd.concat((subset['Minor #1'], subset['Minor #2'])).unique()

In [5]:
def minor_search(major1, major2, major_subset):
    tabs = []    
    for minor1 in all_minors:
        if minor1 is np.nan:
            cnt = major_subset[(major_subset['Minor #1'].isna()) & (major_subset['Minor #2'].isna())].shape[0]
            if cnt:
                tabs.append({
                'major1': major1,
                'major2': 'None' if isinstance(major2, float) else major2,
                'minor1': 'None',
                'minor2': 'None',
                'cnt': cnt,
                })
                continue
        if minor1 in (major1, major2):
            continue
            
        for minor2 in all_minors:
            if minor2 is np.nan:
                if minor2 in (major1, major2):
                    continue
                
                cnt = major_subset[(major_subset['Minor #1'] == minor1) & (major_subset['Minor #2'].isna())].shape[0]
            else:
                cnt = major_subset[((major_subset['Minor #1'] == minor1) & (major_subset['Minor #2'] == minor2)) | \
                                   ((major_subset['Minor #1'] == minor2) & (major_subset['Minor #2'] == minor1))].shape[0]
                    
            if cnt:
                tabs.append({
                    'major1': major1,
                    'major2': 'None' if isinstance(major2, float) else major2,
                    'minor1': minor1,
                    'minor2': 'None' if minor2 is np.nan else minor2,
                    'cnt': cnt
                })
    return tabs

In [6]:
cross_tab = []
for major1 in tqdm(all_majors, desc='Major 1'):
    if major1 is np.nan:
        continue
        
    major2_subsets = []
    for major2 in all_majors:
        if major2 is np.nan:
            major_specific = (subset['Major #1'] == major1) & (subset['Major #2'].isna())
        else:
            major_specific = ((subset['Major #1'] == major1) & (subset['Major #2'] == major2)) | \
                             ((subset['Major #1'] == major2) & (subset['Major #2'] == major1))
        major2_subsets.append(subset[major_specific])
    
    with futures.ProcessPoolExecutor() as executor:
        cross_tab.append(list(tqdm(executor.map(minor_search, itertools.repeat(major1), all_majors, major2_subsets),
                                   total=len(all_majors), desc='Major 2', leave=False)))

Major 1:   0%|          | 0/162 [00:00<?, ?it/s]

Major 2:   0%|          | 0/162 [00:00<?, ?it/s]

Major 2:   0%|          | 0/162 [00:00<?, ?it/s]

Major 2:   0%|          | 0/162 [00:00<?, ?it/s]

Major 2:   0%|          | 0/162 [00:00<?, ?it/s]

...

Major 2:   0%|          | 0/162 [00:00<?, ?it/s]

Major 2:   0%|          | 0/162 [00:00<?, ?it/s]

Major 2:   0%|          | 0/162 [00:00<?, ?it/s]

Major 2:   0%|          | 0/162 [00:00<?, ?it/s]

KeyboardInterrupt: 